# In this task you will play around with character level n-grams, one-hot encoding, and tf-idf.

# Data

In [60]:
corpus = {0: "This Is a Simple sentence", 
          1: "this is another simple one, all lower case..", 
          2: "Hi! How are you? Please update to this version 1.1.1",
          3: "please update to this version 1.1.2 hi! how are you?"}

# Extract n-grams

In [61]:
# Lower/upper case?
allLowerCase=True
# Do you use words or charcters n-grams?
# What delimiters would you use?
delimiters = r'[!? ]' 
# How do you treat digits?
convertDigitsToAsterisk=True
# How long is the n-gram?
n=4

# Character level n-grams

In [62]:
import re

ngram2df = dict()
ngram2tf = dict()
ngram2index = dict()
for sentence_id in corpus.keys(): 
    sentence=corpus[sentence_id]
    if convertDigitsToAsterisk:
        sentence = re.sub(r'[0-9]', '*', sentence)
    if allLowerCase:
        sentence = sentence.lower()
    for word in re.split(delimiters,sentence):
        # Thanks to Shaked and Royee for fixing this bug :-)
        i=n
        while i<=len(word):
            if i>=n:
                token = word[i-n:i]            
                if not(token in ngram2df.keys()):
                    ngram2tf[token] = [0]*len(corpus.keys())
                    ngram2df[token]=set()
                    ngram2index[token] = len(ngram2index.keys())
                ngram2tf[token][sentence_id]+=1
                ngram2df[token].add(sentence_id)
            i+=1
print('Num of ngrams:')
print(len(ngram2tf))

Num of ngrams:
31


## Q: How different values affect the amount of ngrams? Play with n and allLowerCase. 

## A:

# Inspect ngram2tf and ngram2df - make sure they make sense..

In [63]:
#ngram2df
ngram2tf

{'this': [1, 1, 1, 1],
 'simp': [1, 1, 0, 0],
 'impl': [1, 1, 0, 0],
 'mple': [1, 1, 0, 0],
 'sent': [1, 0, 0, 0],
 'ente': [1, 0, 0, 0],
 'nten': [1, 0, 0, 0],
 'tenc': [1, 0, 0, 0],
 'ence': [1, 0, 0, 0],
 'anot': [0, 1, 0, 0],
 'noth': [0, 1, 0, 0],
 'othe': [0, 1, 0, 0],
 'ther': [0, 1, 0, 0],
 'one,': [0, 1, 0, 0],
 'lowe': [0, 1, 0, 0],
 'ower': [0, 1, 0, 0],
 'case': [0, 1, 0, 0],
 'ase.': [0, 1, 0, 0],
 'se..': [0, 1, 0, 0],
 'plea': [0, 0, 1, 1],
 'leas': [0, 0, 1, 1],
 'ease': [0, 0, 1, 1],
 'upda': [0, 0, 1, 1],
 'pdat': [0, 0, 1, 1],
 'date': [0, 0, 1, 1],
 'vers': [0, 0, 1, 1],
 'ersi': [0, 0, 1, 1],
 'rsio': [0, 0, 1, 1],
 'sion': [0, 0, 1, 1],
 '*.*.': [0, 0, 1, 1],
 '.*.*': [0, 0, 1, 1]}

# Build one-hot vectors

In [64]:
sentence2ngrams = dict()
for sentence_id in corpus.keys():
    sentence=corpus[sentence_id]
    if convertDigitsToAsterisk:
        sentence = re.sub(r'[0-9]', '*', sentence)
    if allLowerCase:
        sentence = sentence.lower()
    sentence2ngrams[sentence_id] = [0]*len(ngram2index)
    for word in re.split(delimiters,sentence):
        i=len(word)-n
        while i<=len(word):
            if i>=n:
                token = word[i-n:i]            
                sentence2ngrams[sentence_id][ngram2index[token]] = len(ngram2df[token])
            i+=1
print(sentence2ngrams)

{0: [4, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [4, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2: [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 3: [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}


# Inspect sentence2ngrams - make sure they make sense.. 
## Q: what does a high number represent? 

## A:

## set the params like below and rerun the above.
allLowerCase=True
delimiters = r'[!? ]' 
convertDigitsToAsterisk=True
n=5

## Q: Note that the last two entires in sentence2ngrams are the same. Why is that?

## A:

# tf-idf

In [65]:
import math
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf
def tfidf(tf,df,corpus_size):
    idf = math.log(float(corpus_size)/float(df),10)
    if idf == 0:
        return 0
    return (float(tf)/idf)

In [66]:
sentence2tfidf = dict()
for sentence_id in corpus.keys():
    sentence=corpus[sentence_id]
    if convertDigitsToAsterisk:
        sentence = re.sub(r'[0-9]', '*', sentence)
    if allLowerCase:
        sentence = sentence.lower()
    sentence2tfidf[sentence_id] = [0]*len(ngram2index)
    for word in re.split(delimiters,sentence):
        i=len(word)-n
        while i<=len(word):
            if i>=n:
                token = word[i-n:i]     
                tf = ngram2tf[token][sentence_id]
                df = len(ngram2df[token])
                sentence2tfidf[sentence_id][ngram2index[token]] = tfidf(tf,df,len(corpus.keys()))
            i+=1
print(sentence2tfidf)

{0: [0, 3.321928094887363, 3.321928094887363, 3.321928094887363, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [0, 3.321928094887363, 3.321928094887363, 3.321928094887363, 0, 0, 0, 0, 0, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 1.6609640474436815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363], 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.321928094887363, 3.3219

## Q: inspect the value assigned to the the tokens "this" and  "another". (to do that you will first need to find their indexes. What are their values? Why are they different?

## A: